In [ ]:
import tokenizers
import os

# For suppressing tensorflow warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [ ]:
bwpt = tokenizers.BertWordPieceTokenizer(vocab_file=None,
                                         add_special_tokens=True,
                                         unk_token='[UNK]',
                                         sep_token='[SEP]',
                                         cls_token='[CLS]',
                                         clean_text=True,
                                         handle_chinese_chars=True,
                                         strip_accents=True,
                                         lowercase=True,
                                         wordpieces_prefix='##')

In [ ]:
data = 'path_to_dataset'

bwpt.train(files=[data],
           vocab_size=30000,
           min_frequency=3,
           limit_alphabet=1000,
           special_tokens=['[PAD]', '[UNK]', '[CLS]', '[MASK]', '[SEP]'])

In [ ]:
bwpt.save('/model/', "name_for_vocab")

In [ ]:
!tail -10 /model/name_for_vocab-vocab.txt

In [ ]:
!cd ../src/
!ls -l

In [ ]:
!python create_pretraining_data.py \
    --input_file=data \
    --output_file='./tf_records/tf_examples.tfrecord' \
    --vocab_file='./model/name_for_vocab-vocab.txt' \
    --do_lower_case=True \
    --max_seq_length=128 \
    --max_predictions_per_seq=20 \
    --masked_lm_prob=0.15 \
    --random_seed=0 \
    --dupe_factor=5

In [ ]:
!python run_pretraining.py \
    --input_file='tf_records/tf_examples.tfrecord' \
#     --init_checkpoint='./model/model.ckpt' \
    --output_dir='./model/' \
    --do_train=True \
    --do_eval=True \
    --bert_config_file='path_to_pretrained_model_config.json_file' \
    --train_batch_size=64 \
    --max_seq_length=128 \
    --max_predictions_per_seq=20 \
    --num_train_steps=1000000 \
    --num_warmup_steps=5 \
    --learning_rate=2e-5 \
    --use_tpu=True \
    --tpu_name=$TPU_NAME

In [ ]:
!python convert_tf_to_pytorch.py \
    --tf_checkpoint_path='./model/model.ckpt' \
    --bert_config_file='path_to_pretrained_model_config.json_file' \
    --pytorch_dump_path='/pytorch_model/pytorch_model.bin'